In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline

df = pd.read_csv('calendar.csv') #reads in the calendar data set
df

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN
...,...,...,...,...
1393565,10208623,2016-12-29,f,NaN
1393566,10208623,2016-12-30,f,NaN
1393567,10208623,2016-12-31,f,NaN
1393568,10208623,2017-01-01,f,NaN


In [22]:
drop_missing_dummy = df #creates a dummy dataframe to find mean of price 

In [23]:
drop_missing_dummy

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN
...,...,...,...,...
1393565,10208623,2016-12-29,f,NaN
1393566,10208623,2016-12-30,f,NaN
1393567,10208623,2016-12-31,f,NaN
1393568,10208623,2017-01-01,f,NaN


In [24]:
drop_missing_dummy = drop_missing_dummy.dropna() #drops rows with missing values

In [25]:
drop_missing_dummy #displays drop_missing_dummy to examine that NaNs are gone

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
9,241032,2016-01-13,t,$85.00
10,241032,2016-01-14,t,$85.00
14,241032,2016-01-18,t,$85.00
...,...,...,...,...
1393207,10208623,2016-01-06,t,$87.00
1393208,10208623,2016-01-07,t,$87.00
1393211,10208623,2016-01-10,t,$87.00
1393212,10208623,2016-01-11,t,$87.00


In [26]:
def remove_dollar_signs(dollar): #gets rid of $ to make data more easy to manage
    if isinstance(dollar, str):
        return(dollar.replace('$', '').replace(',', ''))
    return(dollar)
drop_missing_dummy['price'] = drop_missing_dummy['price'].apply(remove_dollar_signs).astype('float')
df['price'] = df['price'].apply(remove_dollar_signs).astype('float')

/Users/sethhamilton/ml/my_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [27]:
drop_missing_dummy['price'].mean() #finds mean of the price column

137.94485855103355

In [28]:
df['price'] = df['price'].fillna(drop_missing_dummy['price'].mean()) #replaces missing prices with the mean price 

In [29]:
df

,listing_id,date,available,price
0,241032,2016-01-04,t,85.000000
1,241032,2016-01-05,t,85.000000
2,241032,2016-01-06,f,137.944859
3,241032,2016-01-07,f,137.944859
4,241032,2016-01-08,f,137.944859
...,...,...,...,...
1393565,10208623,2016-12-29,f,137.944859
1393566,10208623,2016-12-30,f,137.944859
1393567,10208623,2016-12-31,f,137.944859
1393568,10208623,2017-01-01,f,137.944859


In [30]:
encode_df = df
encode_df['year'] = pd.DatetimeIndex(encode_df['date']).year #strips year and month only from the date column
encode_df['month'] = pd.DatetimeIndex(encode_df['date']).month
encode_df.replace(['f','t'], [0,1], inplace = True) #replaces t with 1

In [31]:
encode_df

,listing_id,date,available,price,year,month
0,241032,2016-01-04,1,85.000000,2016,1
1,241032,2016-01-05,1,85.000000,2016,1
2,241032,2016-01-06,0,137.944859,2016,1
3,241032,2016-01-07,0,137.944859,2016,1
4,241032,2016-01-08,0,137.944859,2016,1
...,...,...,...,...,...,...
1393565,10208623,2016-12-29,0,137.944859,2016,12
1393566,10208623,2016-12-30,0,137.944859,2016,12
1393567,10208623,2016-12-31,0,137.944859,2016,12
1393568,10208623,2017-01-01,0,137.944859,2017,1


In [32]:
encode_df = encode_df.drop(columns=['listing_id', 'date', 'available']) #removes the old date column, listing_id column, and available column

In [33]:
encode_df

,price,year,month
0,85.000000,2016,1
1,85.000000,2016,1
2,137.944859,2016,1
3,137.944859,2016,1
4,137.944859,2016,1
...,...,...,...
1393565,137.944859,2016,12
1393566,137.944859,2016,12
1393567,137.944859,2016,12
1393568,137.944859,2017,1


In [34]:
def linear_model(encode_df, test_size=.3, rand_state=42): #function to split explanatory and response variables, split test/train, and predict price of home
    
    X = encode_df.drop(['price'], axis=1)
    y = encode_df['price']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42) 

    lm_model = LinearRegression(normalize=True) # Instantiate
    lm_model.fit(X_train, y_train) #Fit
        
    #Predict and score the model
    y_test_preds = lm_model.predict(X_test) 
    y_train_preds = lm_model.predict(X_train)
    "The r-squared score for the model using only quantitative variables was {} on {} values.".format(r2_score(y_test, y_test_preds), len(y_test))
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test


#Test your function with the above dataset
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = linear_model(encode_df, 'price')

In [35]:
#Print training and testing score
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))

The rsquared on the training data was 0.0011503143785299708.  The rsquared on the test data was 0.0012762655278905077.
